In [2]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

'''g - growth 
s - switching
b - branching
r - retracting
d - dieing
n - new'''

'''
g + b + s = 1

'''
final_csv = ''
g_d_rate = 1
angle = 0.523599
population = ['g', 's', 'b']
time_to_eat_food = 1
max_nodes = 60
epsilon = 0.001

def find_childless(nodes):
    childless_nodes_key = []
    for i in nodes:
        if len(nodes[i]['children']) == 0:
            childless_nodes_key.append(i)
    return childless_nodes_key

def find_node_parent(node_id, nodes):
    for i in nodes:
        if node_id in nodes[i]['children']:
            return nodes[i]

def check_node_near_food(node, food):
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) <= food[i]['r']:
            food[i]['count'] += 1
            type_ret = food[i]['type']
            if (food[i]['count'] >= time_to_eat_food):
                del food[i]
            return [True, type_ret, food]
    return [False, False, food]

def delete_children_nodes(node_id, nodes):
    for i in nodes:
        if i in nodes[i]['children']:
            nodes[i]['children'].remove(i)
    return nodes


def find_closest_food(node, food):
    max_val = float('inf')
    closest_food = {'cord':np.array([0,0,0]), 'r':1, 'type':True, 'count': 0}
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) < max_val:
            closest_food = food[i]
    return closest_food

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def create_new_node(prent_node_id, nodes, food):
    closest_food = find_closest_food(nodes[prent_node_id], food)
    axis = nodes[prent_node_id]['cord']
    v = -axis + closest_food['cord']
    v = (v/(np.linalg.norm(v)+epsilon))*g_d_rate
    v_1 = np.dot(rotation_matrix(axis, angle), np.array([0,0,0]))
    v_2 = np.dot(rotation_matrix(axis, -angle), v)
    #axis + g_d_rate*np.array([random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]), g_d_rate
    max_key = max(list(nodes.keys()))
    print(closest_food)
    nodes[max_key+1] = {
        'cord': axis + g_d_rate*((-axis + closest_food['cord'])/(epsilon+np.linalg.norm(-axis + closest_food['cord']))),
        'children': [],
        'state': 'g'
    }
    nodes[max_key+2] = {
        'cord': axis + g_d_rate*((-axis + closest_food['cord'])/(epsilon+np.linalg.norm(-axis + closest_food['cord']))),
        'children': [],
        'state': 'g'
    }
    nodes[prent_node_id]['children'].extend([max_key+1, max_key+2])
    return nodes

def plot_values(nodes, t, final_csv):
    for key in nodes:
        if len(nodes[key]['children']) >= 1:
            child_0 = nodes[key]['children'][0]
            if child_0 in nodes:
                final_csv += str(nodes[key]['cord'][0]) + ','+ str(nodes[key]['cord'][1]) + ','+ str(nodes[key]['cord'][2]) + ','+ str(nodes[child_0]['cord'][0]) + ',' + str(nodes[child_0]['cord'][1]) + ',' + str(nodes[child_0]['cord'][2]) + ','+ str(t) + '\n'
        if len(nodes[key]['children']) == 2:
            child_1 = nodes[key]['children'][1]
            if child_1 in nodes:
                final_csv += str(nodes[key]['cord'][0]) + ',' + str(nodes[key]['cord'][1]) + ',' + str(nodes[key]['cord'][2]) + ','+ str(nodes[child_1]['cord'][0]) + ',' + str(nodes[child_1]['cord'][1]) + ',' + str(nodes[child_1]['cord'][2]) + ','+ str(t) + '\n'
    return final_csv

def get_csv(t, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu, food):
    final_csv= ''
    nodes = {}
    nodes[0] = {
        'cord': np.array([1, 4, 0]),
        'children': [],
        'state': 'g'
    }
    for j in range(t):
        childless_nodes_key = find_childless(nodes)
        for k, i in enumerate(childless_nodes_key):
            if nodes[i]['state'] == 'r':
                parent = find_node_parent(i, nodes)['cord']
                if np.linalg.norm(parent - nodes[i]['cord']) < g_d_rate:
                    print()
                    del nodes[i]
                    nodes = delete_children_nodes(i, nodes)
                else:
                    nodes[i]['cord'] = nodes[i]['cord'] - g_d_rate
            else:
                is_near_food = check_node_near_food(nodes[i], food)
                food = is_near_food[2]
                if is_near_food[0] and is_near_food[1]:
                    weights = p_pos
                elif is_near_food[0] and (is_near_food[1] == False):
                    weights = p_neg
                else:
                    weights = p_neu
                new_state = random.choices(population, weights=weights, k=1)[0]
                if new_state == 's':
                    nodes[i]['state'] = 'r'
                else:
                    nodes[i]['state'] = new_state
                    if new_state == 'g':
                        parent = find_node_parent(i, nodes)['cord']
                        nodes[i]['cord'] = parent + (-parent + ((nodes[i]['cord'] + g_d_rate)))
                    elif new_state == 'b' and len(nodes) < max_nodes:
                        print("branching")
                        nodes = create_new_node(i, nodes, food)
                    else:
                        new_state = random.choices(population, weights=[0.5, 0.5, 0], k=1)[0]
                        if new_state == 's':
                            nodes[i]['state'] = 'r'
                        else:
                            nodes[i]['state'] = new_state
                            if new_state == 'g':
                                parent = find_node_parent(i, nodes)['cord']
                                nodes[i]['cord'] = parent + (-parent + ((nodes[i]['cord'] + g_d_rate)))
        final_csv = plot_values(nodes, j, final_csv)
    text_file = open("res.txt", "w")
    n = text_file.write(final_csv)
    text_file.close()

In [183]:
t = 1000
#['g', 's', 'b']
p_neg = [0.3, 0.2, 0.5]
p_pos = [0.3, 0.2, 0.5]
p_neu = [0.2, 0.2, 0.5]
b_pos = 1
b_neg = 1
b_neu = 1
food = [{'cord':np.array([10,1,0]), 'r':3, 'type':True, 'count': 0}, 
        {'cord':np.array([9,7,0]), 'r': 3, 'type':False, 'count': 0},
        {'cord':np.array([6,8,0]), 'r': 3, 'type':False, 'count': 0},
        {'cord':np.array([2,3,0]), 'r': 3, 'type':False, 'count': 0},
        {'cord':np.array([3,5,0]), 'r': 3, 'type':True, 'count': 0},
        {'cord':np.array([1,4,0]), 'r':3, 'type':False, 'count': 0}]
get_csv(t, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu, food)

branching
{'cord': array([1, 4, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([1, 4, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}

branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}

branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'count': 0}
branching
{'cord': array([6, 8, 0]), 'r': 3, 'type': False, 'c

In [65]:
v = [3, 5, 0]
axis = [4, 4, 1]
theta = 1.2 

print(np.dot(rotation_matrix(axis, theta), v)) 
# [ 2.74911638  4.77180932  1.91629719]

[2.74911638 4.77180932 1.91629719]


In [11]:
np.cos(20)

0.40808206181339196

In [40]:
max(list(nodes.keys()))

1

In [108]:
import random
import numpy as np

time_to_eat = 10
population = ['g', 's', 'b']
g_r = 0.6
max_nodes = 40

def plot_values(nodes, t, final_csv):
    for node in nodes:
        node_2 = random.randint(0, len(node) - 1)
        node_3 = random.randint(0, len(node) - 1)
        final_csv += str(node['cord'][0]) + ','+ str(node['cord'][1]) + ','+ str(node['cord'][2]) + ','+ str(nodes[node_2]['cord'][0]) + ',' + str(nodes[node_2]['cord'][1]) + ',' + str(nodes[node_2]['cord'][2]) + ','+ str(t) + '\n'
        final_csv += str(node['cord'][0]) + ','+ str(node['cord'][1]) + ','+ str(node['cord'][2]) + ','+ str(nodes[node_3]['cord'][0]) + ',' + str(nodes[node_3]['cord'][1]) + ',' + str(nodes[node_3]['cord'][2]) + ','+ str(t) + '\n'
    return final_csv

def plot_food(food, t, final_food_csv):
    for f in food:
        final_food_csv += str(f['cord'][0]) + ',' + str(f['cord'][1]) + ',' + str(f['cord'][2]) + ',' + str(f['r']) + ',' + str(t) + '\n'
    return final_food_csv

def check_node_near_food(node, food):
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) <= food[i]['r']:
            food[i]['count'] = food[i]['count'] + 1
            type_ret = food[i]['type']
            if (food[i]['count'] >= time_to_eat):
                del food[i]
            return [True, type_ret, food]
    return [False, False, food]

def find_closest_food(node, food):
    max_val = float('inf')
    closest_food = {'cord':np.array([0,0,0]), 'r':1, 'type':True, 'count': 0}
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) < max_val:
            closest_food = food[i]
    return closest_food

def init_points(n, init_r):
    nodes = []
    for i in range(n):
        theta = 2*np.pi*random.random()
        phi = 2*np.pi*random.random()
        r = init_r*random.random()
        x = r*np.cos(theta)
        y = r*np.sin(theta)
        z = r*np.cos(phi)
        random_point = np.array([x, y, z])
        nodes.append({'cord': np.array([x, y, z]), 'state': 'g'})
    return nodes

def init_food(n, min_bound, max_bound, r_range):
    food = []
    for i in range(n):
        r = 2*r_range*random.random()
        x = random.uniform(min_bound, max_bound)
        y = random.uniform(min_bound, max_bound)
        z = random.uniform(min_bound, max_bound)
        food.append({'cord': np.array([x, y, z]), 'count': 0, 'r': r, 'type': True})
    return food

def generate_cycles(nodes, food, cycles, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu):
    final_csv = ''
    final_food_csv = ''
    for t in range(cycles):
        for i, node in enumerate(nodes):
            new_nodes = []
            is_near_food = check_node_near_food(nodes[i], food)
            food = is_near_food[2]
            if is_near_food[0] and is_near_food[1]:
                weights = p_pos
            elif is_near_food[0] and (is_near_food[1] == False):
                weights = p_neg
            else:
                weights = p_neu
            new_state = random.choices(population, weights=weights, k=1)[0]
            node['state'] = new_state
            closest_food = find_closest_food(node, food)
            if new_state == 'g':
                nodes[i]['cord'] = node['cord'] + g_r*(-node['cord'] + closest_food['cord']) + 2*random.random()
            if new_state == 'b' and len(node) < max_nodes:
                cord_1 = node['cord'] + g_r*(-node['cord'] + closest_food['cord']) + 2*random.random()
                cord_2 = node['cord'] + g_r*(-node['cord'] + closest_food['cord']) + 2*random.random()
                new_node_1 = {'cord': cord_1, 'state': 'g'}
                new_node_2 = {'cord': cord_1, 'state': 'g'}
                new_nodes.append(new_node_1)
                new_nodes.append(new_node_2)
        for i, node in enumerate(nodes):
            if node['state'] == 's':
                del nodes[i]
        print(len(food))
        print(len(nodes))
        print('----')
        nodes = nodes + new_nodes
        final_csv = plot_values(nodes, t, final_csv)
        final_food_csv = plot_food(food, t, final_food_csv)
    text_file = open("res.txt", "w")
    n = text_file.write(final_csv)
    text_file.close()
    
    text_file = open("food_res.txt", "w")
    n = text_file.write(final_food_csv)
    text_file.close()

cycles = 200
#['g', 's', 'b']
p_neg = [0.3, 0.1, 0.6]
p_pos = [0.3, 0.1, 0.6]
p_neu = [0.3, 0.1, 0.6]
b_pos = 1
b_neg = 1
b_neu = 1
nodes = init_points(5,1)
food = init_food(20, -20, 20, 10)
generate_cycles(nodes, food, cycles, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu)

20
4
----
20
4
----
20
3
----
19
3
----
19
5
----
19
7
----
17
8
----
17
9
----
16
10
----
16
9
----
15
8
----
14
10
----
14
10
----
13
12
----
12
10
----
12
11
----
12
13
----
12
15
----
11
16
----
10
15
----
10
16
----
10
15
----
10
14
----
9
13
----
9
13
----
8
12
----
8
14
----
8
14
----
8
15
----
7
16
----
7
16
----
5
16
----
5
13
----
5
14
----
5
16
----
5
15
----
5
15
----
5
13
----
5
11
----
5
10
----
5
12
----
5
14
----
5
15
----
5
14
----
5
15
----
5
14
----
5
14
----
4
11
----
4
12
----
4
12
----
4
12
----
4
12
----
4
10
----
4
8
----
4
9
----
4
10
----
4
10
----
4
11
----
4
12
----
4
13
----
4
12
----
3
13
----
3
11
----
3
13
----
3
13
----
3
13
----
3
13
----
3
14
----
3
16
----
3
18
----
3
16
----
3
16
----
3
14
----
3
14
----
3
14
----
3
13
----
3
12
----
3
12
----
3
12
----
3
11
----
3
12
----
3
14
----
3
15
----
3
15
----
3
16
----
3
15
----
3
16
----
3
17
----
3
13
----
3
12
----
3
10
----
3
10
----
3
11
----
3
9
----
3
9
----
3
8
----
3
7
----
3
7
----
3
7
----
3
9
-